In [1]:
from algs_lib import *


In [35]:
def clip_to_threshold(vec, c):
    curr_norm = np.linalg.norm(vec)
#     print(curr_norm, c)
    if curr_norm <= c:
        return vec
#     print('reached')
    clip_ratio = c / curr_norm
    return [vec[i]*clip_ratio for i in range(len(vec))]

In [36]:
def add_noise(scale):
    return np.random.laplace(0, scale)
# global sensitivity is C/n i think?
# so scale should be (C/n) / \epsilon per elem?

In [37]:
def calc_posterior(mi, prior=0.5, prec = 100000):
    test_vals = [x / prec for x in range(1, prec)]
    max_t = None
    for t in test_vals:
        if t*np.log(t/prior)+(1-t)*np.log((1-t)/(1-prior)) <= mi:
            if  max_t is None or t > max_t:
                max_t = t
    return max_t

def dp_epsilon_to_posterior_success(epsilon):
    return 1 - 1./(1+np.exp(epsilon))

def dp_ps_to_epsilon(ps):
    return np.log(ps / (1-ps))

In [38]:
calc_posterior(1.)

0.99999

In [6]:
def hybrid_noise_mean_ind(train_x, train_y, subsample_rate, num_classes,
    eta, regularize=None, num_trees=None, tree_depth = None, max_mi = 0.5, num_dims = None):

    if num_classes is None:
        num_classes = len(set(train_y))

    assert subsample_rate >= num_classes

    s1 = None # only relevant for PCA
    
    max_noises = {}
    
    for ind in range(len(train_x)):
        print(f'ind is {ind}')
        curr_est = None
        converged = False
        curr_trial = 0
        est_y = {}
        prev_ests = None

        removed_train_x = np.delete(train_x, ind, 0)
        removed_train_y = np.delete(train_y, ind, 0)
        while not converged:
            shuffled_x, shuffled_y = shuffle(removed_train_x, removed_train_y)

            shuffled_x, shuffled_y = get_samples_safe(shuffled_x, shuffled_y, num_classes, subsample_rate)
            
            added_x = copy.deepcopy(shuffled_x)
            added_y = copy.deepcopy(shuffled_y)
            added_x[0] = train_x[ind]
            added_y[0] = train_y[ind]

            output_orig = np.average(shuffled_x, axis=0)
            output_new = np.average(added_x, axis=0)
            output = (output_orig - output_new)**2

            for ind in range(len(output)):
                if ind not in est_y:
                    est_y[ind] = []
                est_y[ind].append(output[ind])

            if curr_trial % 10 == 0:
                if prev_ests is None:
                    prev_ests = {}
                    for ind in est_y:
                        prev_ests[ind] = np.average(est_y[ind])
                else:
                    converged = True
                    for ind in est_y:
                        if abs(np.average(est_y[ind]) - prev_ests[ind]) > eta:
                            converged = False
                    if not converged:
                        for ind in est_y:
                            prev_ests[ind] = np.average(est_y[ind])
            curr_trial += 1
        fin_var = {ind: np.average(est_y[ind]) for ind in est_y}

        noise = {}
        sqrt_total_var = sum([fin_var[x]**0.5 for x in fin_var])
        for ind in fin_var:
            noise[ind] = 1./(2*max_mi) * fin_var[ind]**0.5 * sqrt_total_var
        for ind in noise:
            if ind not in max_noises or max_noises[ind] < noise[ind]:
                max_noises[ind] = noise[ind]
    return max_noises

def hybrid_noise_mean(train_x, train_y, subsample_rate, num_classes,
    eta, regularize=None, num_trees=None, tree_depth = None, max_mi = 0.5, num_dims = None):

    sec_v = max_mi / 2
    sec_beta = max_mi - sec_v
    avg_dist = 0.
    curr_est = None
    converged = False
    curr_trial = 0

    if num_classes is None:
        num_classes = len(set(train_y))

    assert subsample_rate >= num_classes

    est_y = {}
    prev_ests = None
    # 10*c*v
    seed = np.random.randint(1, 100000)

    s1 = None # only relevant for PCA
    while not converged:
        shuffled_x, shuffled_y = shuffle(train_x, train_y)
        
        shuffled_x, shuffled_y = get_samples_safe(shuffled_x, shuffled_y, num_classes, subsample_rate)
        
        output = np.average(shuffled_x, axis=0)

        for ind in range(len(output)):
            if ind not in est_y:
                est_y[ind] = []
            est_y[ind].append(output[ind])

        if curr_trial % 10 == 0:
            if curr_trial % 100 == 0:
                print(f'curr trial is {curr_trial}')
            if prev_ests is None:
                prev_ests = {}
                for ind in est_y:
                    prev_ests[ind] = np.var(est_y[ind])
            else:
                converged = True
                for ind in est_y:
                    if abs(np.var(est_y[ind]) - prev_ests[ind]) > eta:
                        converged = False
                if not converged:
                    for ind in est_y:
                        prev_ests[ind] = np.var(est_y[ind])
        curr_trial += 1
    fin_var = {ind: np.var(est_y[ind]) for ind in est_y}

    noise = {}
    sqrt_total_var = sum([fin_var[x]**0.5 for x in fin_var])
    for ind in fin_var:
        noise[ind] = 1./(2*max_mi) * fin_var[ind]**0.5 * sqrt_total_var
    return noise


In [7]:
mi_range = [0.25, 1/16., 0.015625]
posterior_success_rates = [calc_posterior(mi) for mi in mi_range]
epsilon_vals = [dp_ps_to_epsilon(ps) for ps in posterior_success_rates]

print(epsilon_vals)
print([x for x in posterior_success_rates])

[1.6426117097961406, 0.7304317044395013, 0.3563228120191924]
[0.83789, 0.6749, 0.58815]


In [8]:
# DP MEAN
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
true_mean = np.average(train_x, axis=0)

dp_dists = {}
num_trials = 1000
for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 17):
        clip_budget = i / 10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        sensitivity = clip_budget / train_len
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len
                released_mean[ind] += add_noise(sensitivity/eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    print(dp_key)
    dp_dists[eps] = avg_dist_dp[dp_key]


15
14
13


In [9]:
dp_dists

{1.6426117097961406: (0.003760138063449314, 0.023983482677675343),
 0.7304317044395013: (0.01164606776365974, 0.050571370871780146),
 0.3563228120191924: (0.02706865690970566, 0.09830260292875961)}

In [10]:
from scipy import stats

In [11]:
# PAC MEAN LOCAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

noise = hybrid_noise_mean_ind(train_x, train_y, subsample_rate, num_classes, 1e-6)


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

ind is 0
ind is 1
ind is 2
ind is 3
ind is 4
ind is 5
ind is 6
ind is 7
ind is 8
ind is 9
ind is 10
ind is 11
ind is 12
ind is 13
ind is 14
ind is 15
ind is 16
ind is 17
ind is 18
ind is 19
ind is 20
ind is 21
ind is 22
ind is 23
ind is 24
ind is 25
ind is 26
ind is 27
ind is 28
ind is 29
ind is 30
ind is 31
ind is 32
ind is 33
ind is 34
ind is 35
ind is 36
ind is 37
ind is 38
ind is 39
ind is 40
ind is 41
ind is 42
ind is 43
ind is 44
ind is 45
ind is 46
ind is 47
ind is 48
ind is 49
ind is 50
ind is 51
ind is 52
ind is 53
ind is 54
ind is 55
ind is 56
ind is 57
ind is 58
ind is 59
ind is 60
ind is 61
ind is 62
ind is 63
ind is 64
ind is 65
ind is 66
ind is 67
ind is 68
ind is 69
ind is 70
ind is 71
ind is 72
ind is 73
ind is 74
ind is 75
ind is 76
ind is 77
ind is 78
ind is 79
ind is 80
ind is 81
ind is 82
ind is 83
ind is 84
ind is 85
ind is 86
ind is 87
ind is 88
ind is 89
ind is 90
ind is 91
ind is 92
ind is 93
ind is 94
ind is 95
ind is 96
ind is 97
ind is 98
ind is 99
{0.25: (0.

In [12]:
# PAC MEAN GLOBAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

noise = hybrid_noise_mean(train_x, train_y, subsample_rate, num_classes, 1e-6)


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

curr trial is 0
curr trial is 100
curr trial is 200
curr trial is 300
curr trial is 400
curr trial is 500
curr trial is 600
{0.25: (0.04437852008118504, 0.045659487509276744, 0.04624406013370445), 0.0625: (0.04329268021870497, 0.058431306954350413, 0.06614603970132628), 0.015625: (0.04475715052910654, 0.1607382686924796, 0.19147731233596976)}


In [39]:
0.191/0.161

1.186335403726708

In [13]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)

In [14]:
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

3.0829053788327507


In [15]:
true_mean

array([0.13895942, 0.22588616, 0.24527079, 0.23605039, 0.39625879,
       0.76737192, 0.13603607, 0.22457098, 0.6261358 , 0.89987628,
       0.76594898, 0.46047303, 0.49334162, 0.37243875, 0.41431112,
       0.910723  ])

In [16]:
# DP MEAN
dp_dists = {}
num_trials = 1000

for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 31):
        clip_budget = i/10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len 
                released_mean[ind] += add_noise(sensitivity / eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [17]:
dp_dists

{1.6426117097961406: (0.00029846261210171886, 0.0010212688867869352),
 0.7304317044395013: (0.00029846261210171886, 0.00222303019683297),
 0.3563228120191924: (0.0009307916495155954, 0.004371703022545186)}

In [18]:
# PAC MEAN LOCAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)

true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)


noise = hybrid_noise_mean_ind(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    print(f'subsampled_dist = {subsampled_dist}')
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    print(avg_dist_pac)
    print('-----')
#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

ind is 0
ind is 1
ind is 2
ind is 3
ind is 4
ind is 5
ind is 6
ind is 7
ind is 8
ind is 9
ind is 10
ind is 11
ind is 12
ind is 13
ind is 14
ind is 15
ind is 16
ind is 17
ind is 18
ind is 19
ind is 20
ind is 21
ind is 22
ind is 23
ind is 24
ind is 25
ind is 26
ind is 27
ind is 28
ind is 29
ind is 30
ind is 31
ind is 32
ind is 33
ind is 34
ind is 35
ind is 36
ind is 37
ind is 38
ind is 39
ind is 40
ind is 41
ind is 42
ind is 43
ind is 44
ind is 45
ind is 46
ind is 47
ind is 48
ind is 49
ind is 50
ind is 51
ind is 52
ind is 53
ind is 54
ind is 55
ind is 56
ind is 57
ind is 58
ind is 59
ind is 60
ind is 61
ind is 62
ind is 63
ind is 64
ind is 65
ind is 66
ind is 67
ind is 68
ind is 69
ind is 70
ind is 71
ind is 72
ind is 73
ind is 74
ind is 75
ind is 76
ind is 77
ind is 78
ind is 79
ind is 80
ind is 81
ind is 82
ind is 83
ind is 84
ind is 85
ind is 86
ind is 87
ind is 88
ind is 89
ind is 90
ind is 91
ind is 92
ind is 93
ind is 94
ind is 95
ind is 96
ind is 97
ind is 98
ind is 99
ind is 100

ind is 762
ind is 763
ind is 764
ind is 765
ind is 766
ind is 767
ind is 768
ind is 769
ind is 770
ind is 771
ind is 772
ind is 773
ind is 774
ind is 775
ind is 776
ind is 777
ind is 778
ind is 779
ind is 780
ind is 781
ind is 782
ind is 783
ind is 784
ind is 785
ind is 786
ind is 787
ind is 788
ind is 789
ind is 790
ind is 791
ind is 792
ind is 793
ind is 794
ind is 795
ind is 796
ind is 797
ind is 798
ind is 799
ind is 800
ind is 801
ind is 802
ind is 803
ind is 804
ind is 805
ind is 806
ind is 807
ind is 808
ind is 809
ind is 810
ind is 811
ind is 812
ind is 813
ind is 814
ind is 815
ind is 816
ind is 817
ind is 818
ind is 819
ind is 820
ind is 821
ind is 822
ind is 823
ind is 824
ind is 825
ind is 826
ind is 827
ind is 828
ind is 829
ind is 830
ind is 831
ind is 832
ind is 833
ind is 834
ind is 835
ind is 836
ind is 837
ind is 838
ind is 839
ind is 840
ind is 841
ind is 842
ind is 843
ind is 844
ind is 845
ind is 846
ind is 847
ind is 848
ind is 849
ind is 850
ind is 851
ind is 852

ind is 1470
ind is 1471
ind is 1472
ind is 1473
ind is 1474
ind is 1475
ind is 1476
ind is 1477
ind is 1478
ind is 1479
ind is 1480
ind is 1481
ind is 1482
ind is 1483
ind is 1484
ind is 1485
ind is 1486
ind is 1487
ind is 1488
ind is 1489
ind is 1490
ind is 1491
ind is 1492
ind is 1493
ind is 1494
ind is 1495
ind is 1496
ind is 1497
ind is 1498
ind is 1499
ind is 1500
ind is 1501
ind is 1502
ind is 1503
ind is 1504
ind is 1505
ind is 1506
ind is 1507
ind is 1508
ind is 1509
ind is 1510
ind is 1511
ind is 1512
ind is 1513
ind is 1514
ind is 1515
ind is 1516
ind is 1517
ind is 1518
ind is 1519
ind is 1520
ind is 1521
ind is 1522
ind is 1523
ind is 1524
ind is 1525
ind is 1526
ind is 1527
ind is 1528
ind is 1529
ind is 1530
ind is 1531
ind is 1532
ind is 1533
ind is 1534
ind is 1535
ind is 1536
ind is 1537
ind is 1538
ind is 1539
ind is 1540
ind is 1541
ind is 1542
ind is 1543
ind is 1544
ind is 1545
ind is 1546
ind is 1547
ind is 1548
ind is 1549
ind is 1550
ind is 1551
ind is 1552
ind 

ind is 2155
ind is 2156
ind is 2157
ind is 2158
ind is 2159
ind is 2160
ind is 2161
ind is 2162
ind is 2163
ind is 2164
ind is 2165
ind is 2166
ind is 2167
ind is 2168
ind is 2169
ind is 2170
ind is 2171
ind is 2172
ind is 2173
ind is 2174
ind is 2175
ind is 2176
ind is 2177
ind is 2178
ind is 2179
ind is 2180
ind is 2181
ind is 2182
ind is 2183
ind is 2184
ind is 2185
ind is 2186
ind is 2187
ind is 2188
ind is 2189
ind is 2190
ind is 2191
ind is 2192
ind is 2193
ind is 2194
ind is 2195
ind is 2196
ind is 2197
ind is 2198
ind is 2199
ind is 2200
ind is 2201
ind is 2202
ind is 2203
ind is 2204
ind is 2205
ind is 2206
ind is 2207
ind is 2208
ind is 2209
ind is 2210
ind is 2211
ind is 2212
ind is 2213
ind is 2214
ind is 2215
ind is 2216
ind is 2217
ind is 2218
ind is 2219
ind is 2220
ind is 2221
ind is 2222
ind is 2223
ind is 2224
ind is 2225
ind is 2226
ind is 2227
ind is 2228
ind is 2229
ind is 2230
ind is 2231
ind is 2232
ind is 2233
ind is 2234
ind is 2235
ind is 2236
ind is 2237
ind 

ind is 2841
ind is 2842
ind is 2843
ind is 2844
ind is 2845
ind is 2846
ind is 2847
ind is 2848
ind is 2849
ind is 2850
ind is 2851
ind is 2852
ind is 2853
ind is 2854
ind is 2855
ind is 2856
ind is 2857
ind is 2858
ind is 2859
ind is 2860
ind is 2861
ind is 2862
ind is 2863
ind is 2864
ind is 2865
ind is 2866
ind is 2867
ind is 2868
ind is 2869
ind is 2870
ind is 2871
ind is 2872
ind is 2873
ind is 2874
ind is 2875
ind is 2876
ind is 2877
ind is 2878
ind is 2879
ind is 2880
ind is 2881
ind is 2882
ind is 2883
ind is 2884
ind is 2885
ind is 2886
ind is 2887
ind is 2888
ind is 2889
ind is 2890
ind is 2891
ind is 2892
ind is 2893
ind is 2894
ind is 2895
ind is 2896
ind is 2897
ind is 2898
ind is 2899
ind is 2900
ind is 2901
ind is 2902
ind is 2903
ind is 2904
ind is 2905
ind is 2906
ind is 2907
ind is 2908
ind is 2909
ind is 2910
ind is 2911
ind is 2912
ind is 2913
ind is 2914
ind is 2915
ind is 2916
ind is 2917
ind is 2918
ind is 2919
ind is 2920
ind is 2921
ind is 2922
ind is 2923
ind 

...

ind is 6279
ind is 6280
ind is 6281
ind is 6282
ind is 6283
ind is 6284
ind is 6285
ind is 6286
ind is 6287
ind is 6288
ind is 6289
ind is 6290
ind is 6291
ind is 6292
ind is 6293
ind is 6294
ind is 6295
ind is 6296
ind is 6297
ind is 6298
ind is 6299
ind is 6300
ind is 6301
ind is 6302
ind is 6303
ind is 6304
ind is 6305
ind is 6306
ind is 6307
ind is 6308
ind is 6309
ind is 6310
ind is 6311
ind is 6312
ind is 6313
ind is 6314
ind is 6315
ind is 6316
ind is 6317
ind is 6318
ind is 6319
ind is 6320
ind is 6321
ind is 6322
ind is 6323
ind is 6324
ind is 6325
ind is 6326
ind is 6327
ind is 6328
ind is 6329
ind is 6330
ind is 6331
ind is 6332
ind is 6333
ind is 6334
ind is 6335
ind is 6336
ind is 6337
ind is 6338
ind is 6339
ind is 6340
ind is 6341
ind is 6342
ind is 6343
ind is 6344
ind is 6345
ind is 6346
ind is 6347
ind is 6348
ind is 6349
ind is 6350
ind is 6351
ind is 6352
ind is 6353
ind is 6354
ind is 6355
ind is 6356
ind is 6357
ind is 6358
ind is 6359
ind is 6360
ind is 6361
ind 

ind is 6968
ind is 6969
ind is 6970
ind is 6971
ind is 6972
ind is 6973
ind is 6974
ind is 6975
ind is 6976
ind is 6977
ind is 6978
ind is 6979
ind is 6980
ind is 6981
ind is 6982
ind is 6983
ind is 6984
ind is 6985
ind is 6986
ind is 6987
ind is 6988
ind is 6989
ind is 6990
ind is 6991
ind is 6992
ind is 6993
ind is 6994
ind is 6995
ind is 6996
ind is 6997
ind is 6998
ind is 6999
ind is 7000
ind is 7001
ind is 7002
ind is 7003
ind is 7004
ind is 7005
ind is 7006
ind is 7007
ind is 7008
ind is 7009
ind is 7010
ind is 7011
ind is 7012
ind is 7013
ind is 7014
ind is 7015
ind is 7016
ind is 7017
ind is 7018
ind is 7019
ind is 7020
ind is 7021
ind is 7022
ind is 7023
ind is 7024
ind is 7025
ind is 7026
ind is 7027
ind is 7028
ind is 7029
ind is 7030
ind is 7031
ind is 7032
ind is 7033
ind is 7034
ind is 7035
ind is 7036
ind is 7037
ind is 7038
ind is 7039
ind is 7040
ind is 7041
ind is 7042
ind is 7043
ind is 7044
ind is 7045
ind is 7046
ind is 7047
ind is 7048
ind is 7049
ind is 7050
ind 

ind is 7654
ind is 7655
ind is 7656
ind is 7657
ind is 7658
ind is 7659
ind is 7660
ind is 7661
ind is 7662
ind is 7663
ind is 7664
ind is 7665
ind is 7666
ind is 7667
ind is 7668
ind is 7669
ind is 7670
ind is 7671
ind is 7672
ind is 7673
ind is 7674
ind is 7675
ind is 7676
ind is 7677
ind is 7678
ind is 7679
ind is 7680
ind is 7681
ind is 7682
ind is 7683
ind is 7684
ind is 7685
ind is 7686
ind is 7687
ind is 7688
ind is 7689
ind is 7690
ind is 7691
ind is 7692
ind is 7693
ind is 7694
ind is 7695
ind is 7696
ind is 7697
ind is 7698
ind is 7699
ind is 7700
ind is 7701
ind is 7702
ind is 7703
ind is 7704
ind is 7705
ind is 7706
ind is 7707
ind is 7708
ind is 7709
ind is 7710
ind is 7711
ind is 7712
ind is 7713
ind is 7714
ind is 7715
ind is 7716
ind is 7717
ind is 7718
ind is 7719
ind is 7720
ind is 7721
ind is 7722
ind is 7723
ind is 7724
ind is 7725
ind is 7726
ind is 7727
ind is 7728
ind is 7729
ind is 7730
ind is 7731
ind is 7732
ind is 7733
ind is 7734
ind is 7735
ind is 7736
ind 

ind is 8340
ind is 8341
ind is 8342
ind is 8343
ind is 8344
ind is 8345
ind is 8346
ind is 8347
ind is 8348
ind is 8349
ind is 8350
ind is 8351
ind is 8352
ind is 8353
ind is 8354
ind is 8355
ind is 8356
ind is 8357
ind is 8358
ind is 8359
ind is 8360
ind is 8361
ind is 8362
ind is 8363
ind is 8364
ind is 8365
ind is 8366
ind is 8367
ind is 8368
ind is 8369
ind is 8370
ind is 8371
ind is 8372
ind is 8373
ind is 8374
ind is 8375
ind is 8376
ind is 8377
ind is 8378
ind is 8379
ind is 8380
ind is 8381
ind is 8382
ind is 8383
ind is 8384
ind is 8385
ind is 8386
ind is 8387
ind is 8388
ind is 8389
ind is 8390
ind is 8391
ind is 8392
ind is 8393
ind is 8394
ind is 8395
ind is 8396
ind is 8397
ind is 8398
ind is 8399
ind is 8400
ind is 8401
ind is 8402
ind is 8403
ind is 8404
ind is 8405
ind is 8406
ind is 8407
ind is 8408
ind is 8409
ind is 8410
ind is 8411
ind is 8412
ind is 8413
ind is 8414
ind is 8415
ind is 8416
ind is 8417
ind is 8418
ind is 8419
ind is 8420
ind is 8421
ind is 8422
ind 

ind is 9026
ind is 9027
ind is 9028
ind is 9029
ind is 9030
ind is 9031
ind is 9032
ind is 9033
ind is 9034
ind is 9035
ind is 9036
ind is 9037
ind is 9038
ind is 9039
ind is 9040
ind is 9041
ind is 9042
ind is 9043
ind is 9044
ind is 9045
ind is 9046
ind is 9047
ind is 9048
ind is 9049
ind is 9050
ind is 9051
ind is 9052
ind is 9053
ind is 9054
ind is 9055
ind is 9056
ind is 9057
ind is 9058
ind is 9059
ind is 9060
ind is 9061
ind is 9062
ind is 9063
ind is 9064
ind is 9065
ind is 9066
ind is 9067
ind is 9068
ind is 9069
ind is 9070
ind is 9071
ind is 9072
ind is 9073
ind is 9074
ind is 9075
ind is 9076
ind is 9077
ind is 9078
ind is 9079
ind is 9080
ind is 9081
ind is 9082
ind is 9083
ind is 9084
ind is 9085
ind is 9086
ind is 9087
ind is 9088
ind is 9089
ind is 9090
ind is 9091
ind is 9092
ind is 9093
ind is 9094
ind is 9095
ind is 9096
ind is 9097
ind is 9098
ind is 9099
ind is 9100
ind is 9101
ind is 9102
ind is 9103
ind is 9104
ind is 9105
ind is 9106
ind is 9107
ind is 9108
ind 

In [19]:
# PAC MEAN GLOBAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)

true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)


noise = hybrid_noise_mean(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    print(f'subsampled_dist = {subsampled_dist}')
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    print(avg_dist_pac)
    print('-----')
#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

curr trial is 0
subsampled_dist = 0.005327442843191195
0.005333651632707698
-----
subsampled_dist = 0.0055277512023112025
0.005626750137096074
-----
subsampled_dist = 0.005430068983593693
0.006914864337075771
-----
{0.25: (0.005327442843191195, 0.005333651632707698, 0.005489552769997568), 0.0625: (0.0055277512023112025, 0.005626750137096074, 0.00556474002625065), 0.015625: (0.005430068983593693, 0.006914864337075771, 0.007885297374603646)}


In [20]:
pac_dists

{0.25: (0.005327442843191195, 0.005333651632707698, 0.005489552769997568),
 0.0625: (0.0055277512023112025, 0.005626750137096074, 0.00556474002625065),
 0.015625: (0.005430068983593693, 0.006914864337075771, 0.007885297374603646)}

In [21]:
pac_dists

{0.25: (0.005327442843191195, 0.005333651632707698, 0.005489552769997568),
 0.0625: (0.0055277512023112025, 0.005626750137096074, 0.00556474002625065),
 0.015625: (0.005430068983593693, 0.006914864337075771, 0.007885297374603646)}

In [22]:
# RICE

In [23]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)

In [24]:
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

2.399557199470374


In [25]:
true_mean

array([0.44713686, 0.49900406, 0.46102121, 0.55540665, 0.64166967,
       0.45641727, 0.44921844])

In [31]:
# DP MEAN
dp_dists = {}
num_trials = 1000

for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 24):
        clip_budget = i / 10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len 
                released_mean[ind] += add_noise(sensitivity / eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [32]:
dp_dists

{1.6426117097961406: (0.00014964886414599332, 0.0017234747657026838),
 0.7304317044395013: (0.0005036889204271803, 0.003778597909724593),
 0.3563228120191924: (0.00174717778612465, 0.007474336838708587)}

In [33]:
# PAC MEAN LOCAL

train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)


noise = hybrid_noise_mean_ind(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000

for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

ind is 0
ind is 1
ind is 2
ind is 3
ind is 4
ind is 5
ind is 6
ind is 7
ind is 8
ind is 9
ind is 10
ind is 11
ind is 12
ind is 13
ind is 14
ind is 15
ind is 16
ind is 17
ind is 18
ind is 19
ind is 20
ind is 21
ind is 22
ind is 23
ind is 24
ind is 25
ind is 26
ind is 27
ind is 28
ind is 29
ind is 30
ind is 31
ind is 32
ind is 33
ind is 34
ind is 35
ind is 36
ind is 37
ind is 38
ind is 39
ind is 40
ind is 41
ind is 42
ind is 43
ind is 44
ind is 45
ind is 46
ind is 47
ind is 48
ind is 49
ind is 50
ind is 51
ind is 52
ind is 53
ind is 54
ind is 55
ind is 56
ind is 57
ind is 58
ind is 59
ind is 60
ind is 61
ind is 62
ind is 63
ind is 64
ind is 65
ind is 66
ind is 67
ind is 68
ind is 69
ind is 70
ind is 71
ind is 72
ind is 73
ind is 74
ind is 75
ind is 76
ind is 77
ind is 78
ind is 79
ind is 80
ind is 81
ind is 82
ind is 83
ind is 84
ind is 85
ind is 86
ind is 87
ind is 88
ind is 89
ind is 90
ind is 91
ind is 92
ind is 93
ind is 94
ind is 95
ind is 96
ind is 97
ind is 98
ind is 99
ind is 100

ind is 764
ind is 765
ind is 766
ind is 767
ind is 768
ind is 769
ind is 770
ind is 771
ind is 772
ind is 773
ind is 774
ind is 775
ind is 776
ind is 777
ind is 778
ind is 779
ind is 780
ind is 781
ind is 782
ind is 783
ind is 784
ind is 785
ind is 786
ind is 787
ind is 788
ind is 789
ind is 790
ind is 791
ind is 792
ind is 793
ind is 794
ind is 795
ind is 796
ind is 797
ind is 798
ind is 799
ind is 800
ind is 801
ind is 802
ind is 803
ind is 804
ind is 805
ind is 806
ind is 807
ind is 808
ind is 809
ind is 810
ind is 811
ind is 812
ind is 813
ind is 814
ind is 815
ind is 816
ind is 817
ind is 818
ind is 819
ind is 820
ind is 821
ind is 822
ind is 823
ind is 824
ind is 825
ind is 826
ind is 827
ind is 828
ind is 829
ind is 830
ind is 831
ind is 832
ind is 833
ind is 834
ind is 835
ind is 836
ind is 837
ind is 838
ind is 839
ind is 840
ind is 841
ind is 842
ind is 843
ind is 844
ind is 845
ind is 846
ind is 847
ind is 848
ind is 849
ind is 850
ind is 851
ind is 852
ind is 853
ind is 854

ind is 1501
ind is 1502
ind is 1503
ind is 1504
ind is 1505
ind is 1506
ind is 1507
ind is 1508
ind is 1509
ind is 1510
ind is 1511
ind is 1512
ind is 1513
ind is 1514
ind is 1515
ind is 1516
ind is 1517
ind is 1518
ind is 1519
ind is 1520
ind is 1521
ind is 1522
ind is 1523
ind is 1524
ind is 1525
ind is 1526
ind is 1527
ind is 1528
ind is 1529
ind is 1530
ind is 1531
ind is 1532
ind is 1533
ind is 1534
ind is 1535
ind is 1536
ind is 1537
ind is 1538
ind is 1539
ind is 1540
ind is 1541
ind is 1542
ind is 1543
ind is 1544
ind is 1545
ind is 1546
ind is 1547
ind is 1548
ind is 1549
ind is 1550
ind is 1551
ind is 1552
ind is 1553
ind is 1554
ind is 1555
ind is 1556
ind is 1557
ind is 1558
ind is 1559
ind is 1560
ind is 1561
ind is 1562
ind is 1563
ind is 1564
ind is 1565
ind is 1566
ind is 1567
ind is 1568
ind is 1569
ind is 1570
ind is 1571
ind is 1572
ind is 1573
ind is 1574
ind is 1575
ind is 1576
ind is 1577
ind is 1578
ind is 1579
ind is 1580
ind is 1581
ind is 1582
ind is 1583
ind 

ind is 2202
ind is 2203
ind is 2204
ind is 2205
ind is 2206
ind is 2207
ind is 2208
ind is 2209
ind is 2210
ind is 2211
ind is 2212
ind is 2213
ind is 2214
ind is 2215
ind is 2216
ind is 2217
ind is 2218
ind is 2219
ind is 2220
ind is 2221
ind is 2222
ind is 2223
ind is 2224
ind is 2225
ind is 2226
ind is 2227
ind is 2228
ind is 2229
ind is 2230
ind is 2231
ind is 2232
ind is 2233
ind is 2234
ind is 2235
ind is 2236
ind is 2237
ind is 2238
ind is 2239
ind is 2240
ind is 2241
ind is 2242
ind is 2243
ind is 2244
ind is 2245
ind is 2246
ind is 2247
ind is 2248
ind is 2249
ind is 2250
ind is 2251
ind is 2252
ind is 2253
ind is 2254
ind is 2255
ind is 2256
ind is 2257
ind is 2258
ind is 2259
ind is 2260
ind is 2261
ind is 2262
ind is 2263
ind is 2264
ind is 2265
ind is 2266
ind is 2267
ind is 2268
ind is 2269
ind is 2270
ind is 2271
ind is 2272
ind is 2273
ind is 2274
ind is 2275
ind is 2276
ind is 2277
ind is 2278
ind is 2279
ind is 2280
ind is 2281
ind is 2282
ind is 2283
ind is 2284
ind 

In [29]:
# PAC MEAN GLOBAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)

true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)


noise = hybrid_noise_mean(train_x, train_y, subsample_rate, num_classes, 1e-6)

pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    print(f'subsampled_dist = {subsampled_dist}')
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials
    print(avg_dist_pac)
    print('-----')
#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

curr trial is 0
subsampled_dist = 0.007740236203163901
0.0077616217254393985
-----
subsampled_dist = 0.00784245216905404
0.008123173169984161
-----
subsampled_dist = 0.007874707709061736
0.011379490644424983
-----
{0.25: (0.007740236203163901, 0.0077616217254393985, 0.007754947082684102), 0.0625: (0.00784245216905404, 0.008123173169984161, 0.008188867166111127), 0.015625: (0.007874707709061736, 0.011379490644424983, 0.01293948864580817)}


In [30]:
pac_dists

{0.25: (0.007740236203163901, 0.0077616217254393985, 0.007754947082684102),
 0.0625: (0.00784245216905404, 0.008123173169984161, 0.008188867166111127),
 0.015625: (0.007874707709061736, 0.011379490644424983, 0.01293948864580817)}